To insert a mathematical formula we use the dollar symbol, as follows:
Euler's identity: $ e^{i \pi} + 1 = 0 $
To isolate and center the formulas and enter in math display mode, we use 2 dollars symbol:
Euler's identity: $$e^{i \pi} + 1 = 0$$

Given : $\pi = 3.14$ , $\alpha = \frac{3\pi}{4}\, rad$
$$
\omega = 2\pi f \\
f = \frac{c}{\lambda}\\
\lambda_0=\theta^2+\delta\\
\Delta\lambda = \frac{1}{\lambda^2}
$$

Given : $\pi = 3.14$ , $\alpha = \frac{3\pi}{4}\, rad$
$$
\omega = 2\pi f \\
f = \frac{c}{\lambda}\\
\lambda_0=\theta^2+\delta\\
\Delta\lambda = \frac{1}{\lambda^2}
$$